## Les agents

Un agent est un système autonome alimenté par un modèle de langage (comme GPT-4) qui prend des décisions sur les actions à entreprendre en fonction des données d'entrée et des instructions programmées.

Fonction :
- Prise de décision : L'agent analyse les données d'entrée et utilise des algorithmes et des modèles pour décider quelle action entreprendre.
- Exécution d'actions : L'agent peut effectuer diverses actions comme répondre à une question, rechercher des informations, ou interagir avec d'autres systèmes.

Un "agent", c'est un LLM avec des "outils" :
 - recherche sur internet,
 - calculatrice,
 - interrogation de pdf (RAG),
 - outil fait maison
 - ...

Le mieux est de faire des agents spécialisés, et de les orchestrer ensemble.

### Des outils

#### Wikipedia

In [ ]:
#%pip install --upgrade --quiet  wikipedia
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
wikipedia.run("Alan Turing")

#### Tavili (recherche internet)

In [ ]:
# pip install -qU langchain-tavily
# Pour une clé d'API : https://www.tavily.com/
from langchain_community.tools.tavily_search import TavilySearchResults

import os
os.environ["TAVILY_API_KEY"] = "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"

search = TavilySearchResults(max_results=2)
search_results = search.invoke("Quel est le temps à Belfort ?")
print(search_results[0]['content'][:100])

### Agents LangChain

#### Exécuteur d'agent (Agent Executor)

L'exécuteur d'agent est un composant ou un système qui orchestre et exécute les actions déterminées par l'agent.

Fonction :
 - Gestion de l'exécution : Il reçoit les décisions de l'agent, exécute les actions correspondantes et gère la transition entre différentes étapes de l'exécution.
 - Traitement des résultats : Il collecte les résultats des actions exécutées et les transmet à l'agent pour de nouvelles décisions ou à l'utilisateur final.

Exemple : Dans un système de recommandation, l'exécuteur d'agent pourrait orchestrer l'appel à différentes API pour recueillir les informations nécessaires (comme les préférences de l'utilisateur et les données sur les produits) et les combiner pour générer une recommandation.

#### Avec outil Tavily

In [ ]:
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.messages import HumanMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_mistralai.chat_models import ChatMistralAI

search = TavilySearchResults(max_results=2)
tools = [search]

llm = ChatMistralAI(model="mistral-large-latest", temperature=0)

prompt = hub.pull("amalnuaimi/react-mistral")
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = agent_executor.invoke(
    {
        'input': "Dois-je prendre un parapluie, sachant que je me rends aujourd'hui et demain à Belfort ?",
        'chat_history': []
    })
print(response['output'])

In [ ]:
#https://smith.langchain.com/hub
prompt.pretty_print()

In [ ]:
# Version OpenAI
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.messages import HumanMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI

search = TavilySearchResults(max_results=2)
tools = [search]

llm = ChatOpenAI(model_name="gpt-4.1")

prompt = hub.pull("hwchase17/react")
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = agent_executor.invoke(
    {
        'input': "Dois-je prendre un parapluie, sachant que je me rends aujourd'hui et demain à Belfort ?"
    })
print(response['output'])

#### Avec outil arXiv

In [ ]:
# pip install arxiv
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain_openai import ChatOpenAI
from langchain_mistralai.chat_models import ChatMistralAI

#llm = ChatOpenAI(temperature=0.0)
llm = ChatMistralAI(model="mistral-large-latest", temperature=0)
tools = load_tools(["arxiv"])

prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "Résume l'article 1605.08386 en français"})

In [ ]:
prompt.pretty_print()

In [ ]:
print(tools[0].name)
print(tools[0].description)

#### Avec Python REPL

In [ ]:
from langchain.agents import create_openai_functions_agent
from langchain_experimental.tools import PythonREPLTool

tools = [PythonREPLTool()]

instructions = """You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.
"""

base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)
agent = create_openai_functions_agent(ChatOpenAI(temperature=0), tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "Quel est le millième nombre de Fibonacci ?"})

In [ ]:
base_prompt.pretty_print()

#### Plusieurs outils

In [ ]:
from langchain.agents import load_tools, create_react_agent
from langchain_openai import OpenAI

llm = OpenAI()
tools = load_tools(["llm-math", "wikipedia"], llm=llm)
prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, 
                               tools=tools, 
                               handle_parsing_errors=True, 
                               verbose=True)

In [ ]:
agent_executor.invoke({'input': "Qu'est-ce que 25% de 300?"})

### Ses propres outils

In [ ]:
from langchain_core.tools import tool

@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiplie deux entiers."""
    return first_int * second_int

print(multiply.name)
print(multiply.description)
print(multiply.args)

In [ ]:
@tool
def add(first_int: int, second_int: int) -> int:
    "Ajoute deux entiers."
    return first_int + second_int

@tool
def exponentiate(base: int, exponent: int) -> int:
    "Calcule la puissance d'un entier donné."
    return base**exponent

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4.1")

tools = [multiply, add, exponentiate]
prompt = hub.pull("hwchase17/openai-tools-agent")
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke(
    {
        "input": "Porter 3 à la puissance 5 et multiplier le résultat par la somme de douze et de trois, puis élever le tout au carré."
    }
)

In [ ]:
prompt.pretty_print()

### Assemblage d'agents

![Texte alternatif](images/promptulate.png "Promptulate")